In [ ]:
# claification PCE
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, precision_score, roc_curve, auc, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier,GradientBoostingClassifier
from joblib import dump
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
# Assuming the data has been uploaded to the session and is available as a pandas DataFrame
# Load the dataset
df = pd.read_csv('data/Data_GCN_prering2alt_fin_fused.csv')

# Function to categorize PCE values
def categorize_pce(pce_value):
    if pce_value > 17:
        return 'A'  # High level
    elif pce_value >= 13:
        return 'B'  # Moderate level
    else:
        return 'C'  # Low level

# Apply categorization to the 'PCE_max(%)' column
df['PCE_Category'] = df['PCE_ave(%)'].apply(categorize_pce)

# Identify the maximum length of features after splitting
max_length = df['Combined_Features'].apply(lambda x: len(x.split(','))).max()

# Function to pad the features with zeros
def pad_features(feature_string, max_length):
    features = np.fromstring(feature_string, sep=',')
    padded_features = np.pad(features, (0, max_length - len(features)), 'constant', constant_values=0)
    return padded_features

# Apply padding to each row in the 'Combined_Features' column
X = np.array([pad_features(x, max_length) for x in df['Combined_Features']])
Y = df['PCE_Category'].values  # Categories are 'A', 'B', 'C'

# Shuffle and split the data into training and test sets
np.random.seed(0)
X, Y = sklearn.utils.shuffle(X, Y)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

# Define the model
# classifier = RandomForestClassifier(n_estimators=100, max_depth=23, random_state=90)
# classifier = ExtraTreesClassifier(n_estimators=80, max_depth=23, random_state=90)
# classifier = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=90)
# classifier = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', random_state=90)
classifier = SVC(kernel='rbf', C=1.0, probability=True, random_state=90)

# Train the model
classifier.fit(X_train, Y_train)

# Predict on test data
Y_pred = classifier.predict(X_test)

# Calculate the probabilities for ROC curve
Y_probs = classifier.predict_proba(X_test)[:, 1]  # Assuming 'A' class (high level) is the positive class

# Compute the ROC curve and AUC score
fpr, tpr, thresholds = roc_curve(Y_test, Y_probs, pos_label='A')
roc_auc = auc(fpr, tpr)

# Evaluate the classifier
accuracy = accuracy_score(Y_test, Y_pred)
precision = precision_score(Y_test, Y_pred, average='weighted')  # Use 'weighted' for imbalanced classes
# classification_rep = classification_report(Y_test, Y_pred)
# conf_matrix = confusion_matrix(Y_test, Y_pred)

accuracy, precision, roc_auc, 